# Additional End of week Exercise - week 2

* I have taken this noot book from another studient , it was shared in the community contributions folder but i make some changes on it and reshared it .  


In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import random
import re
import base64
import sqlite3
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display
import speech_recognition as sr #requires pip install speechrecognition AND pip install pyaudio
import tempfile


In [2]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
else:
    print("OpenAI API Key? As if!")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

DB = "prices.db"

OpenAI API Key exists and begins with sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so. "
system_message += "You can book flights directly. "
system_message += "You can generate beautiful artistic renditions of the cities we fly to."

In [4]:
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [5]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [6]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [7]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [8]:
get_ticket_price("paris")

DATABASE TOOL CALLED: Getting price for paris


'Ticket price to paris is $899.0'

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}



[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [10]:
# travel booking function

def book_flight(destination_city):
    booking_code = ''.join(random.choice('0123456789BCDFXYZ') for i in range(2)) + ''.join(random.choice('012346789HIJKLMNOPQRS') for i in range(2)) + ''.join(random.choice('0123456789GHIJKLMNUOP') for i in range(2))
    print(f"Booking code {booking_code} generated for flight to {destination_city}.")
    
    return booking_code

In [11]:
booking_function = {
    "name": "book_flight",
    "description": "Call this whenever you have to book a flight. Give it the destination city and you will get a booking code. Tell the customer \
that the flight is booked and give them the booking code obtained through this function. Never give any other codes to the customer.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book their flight to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [12]:
# verify if booking code is valid (i.e. follows the pattern)

def check_code(code):
    valid = "valid" if re.match("^[0123456789BCDFXYZ]{2}[012346789HIJKLMNOPQRS]{2}[0123456789GHIJKLMNUOP]{2}$", code) is not None else "not valid"
    print(f"Code checker called for code {code}, which is {valid}.")
    return re.match("^[0123456789BCDFXYZ]{2}[012346789HIJKLMNOPQRS]{2}[0123456789GHIJKLMNUOP]{2}$", code) is not None

In [23]:
codecheck_function = {
    "name": "check_code",
    "description": "Call this whenever you need to verify if a booking code for a flight (also called 'flight code', 'booking reference', \
or variations thereof) is valid.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "The code that you or the user needs to verify",
            },
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book their flight to",
            },
        },
        "required": ["code"],
        "additionalProperties": False
    }
}

In [14]:
# make a nice preview of the travel destination

def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [15]:
# list of tools

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}, 
    {"type": "function", "function": codecheck_function} 
    ]

tools    

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'book_flight',
   'description': 'Call this whenever you have to book a flight. Give it the destination city and you will get a booking code. Tell the customer that the flight is booked and give them the booking code obtained through this function. Never give any other codes to the customer.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to

In [16]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

In [22]:
def speech_to_text(audio_file):
    """Convert speech to text using OpenAI Whisper"""
    if audio_file is None:
        return ""
    
    with open(audio_file, "rb") as f:
        transcription = openai.audio.transcriptions.create(
            model="gpt-4o-mini-transcribe",  # or "whisper-1"
            file=f
        )
    return transcription.text

In [24]:
def handle_tool_calls_and_return_cities(message):
    responses = []
    cities = []
    
    for tool_call in message.tool_calls:
        name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        # Handle get_ticket_price
        if name == "get_ticket_price":
            city = arguments.get("destination_city")
            cities.append(city)
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })

        # Handle book_flight
        elif name == "book_flight":
            city = arguments.get("destination_city")
            cities.append(city)
            booking_code = book_flight(city)
            responses.append({
                "role": "tool",
                "content": f"Flight booked to {city}. Booking code: {booking_code}",
                "tool_call_id": tool_call.id
            })

        # Handle check_code
        elif name == "check_code":
            city = arguments.get("destination_city")
            cities.append(city)
            code = arguments.get("code")
            is_valid = check_code(code)
            responses.append({
                "role": "tool",
                "content": f"The booking code {code} is {'valid' if is_valid else 'invalid'}.",
                "tool_call_id": tool_call.id
            })

    return responses, cities


In [25]:
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    cities = []
    image = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses, cities = handle_tool_calls_and_return_cities(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)

    if cities:
        image = artist(cities[0])
    
    return history, voice, image

In [ ]:
# Callbacks (along with the chat() function above)

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        mic = gr.Audio(label="🎤 Speak to your AI Assistant", type="filepath")
# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )

    # 🎤 When you speak into the mic
    mic.change(
        lambda audio, hist: ("", hist + [{"role": "user", "content": speech_to_text(audio)}]),
        inputs=[mic, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True, auth=("AHMED", "0511"))

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware

DATABASE TOOL CALLED: Getting price for Tokyo
Booking code F8RR17 generated for flight to Tokyo.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "d:\programing\ML_and_DL\04-NLP\udemy_mastering_AI_LLMs\llm_engineering\.venv\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware

Code checker called for code F8RR17, which is valid.
Code checker called for code F8RR17, which is valid.
